# Head

In [ ]:
%pip install mambapy

In [1]:
import torch, transformers, importlib
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import Mamba, MambaVAE, Dataset

# Lab

In [70]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Sunday.", "Today is."], 
                      return_tensors="pt", 
                      truncation=True, 
                      max_length=10, 
                      padding="max_length") #.to('cuda')
ones = torch.ones((inputs['attention_mask'].size(0), 1))
inputs['attention_mask'] = torch.cat([ones, inputs['attention_mask'][:, :-1]], dim=1)

In [89]:
importlib.reload(MambaVAE)

model = MambaVAE.MambaVAE()
model.eval()
model(**inputs, do_sample=False)

(tensor(27.2230, grad_fn=<DivBackward0>),
 tensor(40509.1094, grad_fn=<MulBackward0>),
 tensor([[[ 15.4702,  -6.4229,  11.7433,  ...,  -6.3040,  -6.8389,  -6.4293],
          [ 13.2942, -49.8134,   4.4575,  ..., -49.5522, -50.0998, -49.6902],
          [  9.3128, -42.6583,   0.5586,  ..., -42.3826, -42.9728, -42.5657],
          ...,
          [  3.7884, -17.8751,   6.2833,  ..., -18.5168, -18.0260, -18.3387],
          [  2.8193, -15.6524,   6.6504,  ..., -16.2095, -15.6207, -16.1042],
          [  0.9937, -11.9810,   6.8434,  ..., -12.4709, -11.7472, -12.3935]],
 
         [[ 15.4759,  -6.3992,  11.7435,  ...,  -6.2799,  -6.8146,  -6.4048],
          [ 13.3054, -49.7845,   4.4584,  ..., -49.5229, -50.0700, -49.6606],
          [  9.3230, -42.6298,   0.5528,  ..., -42.3538, -42.9437, -42.5369],
          ...,
          [  3.7884, -17.8751,   6.2833,  ..., -18.5168, -18.0260, -18.3387],
          [  2.8193, -15.6524,   6.6504,  ..., -16.2095, -15.6207, -16.1042],
          [  0.9937, -

In [65]:
importlib.reload(Mamba)
import Mamba

config = MambaConfig.from_pretrained("state-spaces/mamba-130m-hf")
config.use_mambapy = True
# config.num_hidden_layers = 12
model1 = Mamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config) #.to('cuda')
model1.eval()

res2 = model1(**inputs, output_ssm_last_states = True)

In [63]:
ssm_last_states = res2.ssm_last_states

In [66]:
res2 = model1(**inputs, inputs_ssm_states=ssm_last_states, output_ssm_last_states = True)
# print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))
# print(res2.ssm_last_states)

In [67]:
torch.abs(res1.ssm_last_states - res2.ssm_last_states).sum()

tensor(0.0430, grad_fn=<SumBackward0>)

In [30]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)
model2.train()

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (norm): MambaRMSNorm(768, eps=1e-05)
        (mixer): MambaMixer(
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm(768, eps=1e-05)
  )
  (lm_head): Linear(in_features=768, out_features=50280, bias=False)
)

# Check same

In [44]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(**inputs).logits - model1(**inputs).logits).abs().sum())

loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
tensor(0., grad_fn=<SumBackward0>)
